### Introduction

Cloud Functions is a **lightweight**, **event-based**, **asynchronous** compute solution that allows you to create small, **single-purpose** functions that respond to cloud events without the need to manage a server or a runtime environment.

In fact, when we use **cloud function** we could just focus on the implementation of the logic that we need to get, like we could use **cloud function** to do interact with `database`, or `pubsub`, or `storage` etc. 


In [ ]:
# first set project
! gcloud config set project cloudtutorial-279003

Updated property [core/project].


In [ ]:
# auth the notebook
from google.colab import auth
auth.authenticate_user()


### Hello Cloud function

First let's make the first hello world function to hear HTTP request to get the request data and return the result with HTTP response.

In [ ]:
# first make a hello_world function
%%writefile main.py

def hello_world(request):
  # convert request into json
  request_json = request.get_json()

  if request.args and 'message' in request.args:
    return request.args.get("message")
  elif request_json and 'message' in request_json:
    return request_json['message']
  else:
    return "hello lu"

Writing main.py


In [ ]:
# after we have already create the main function, let's try to deploy this function 
! gcloud functions deploy hello_world --runtime python37 --trigger-http --allow-unauthenticated

availableMemoryMb: 256
entryPoint: hello_world
httpsTrigger:
  url: https://us-central1-cloudtutorial-279003.cloudfunctions.net/hello_world
ingressSettings: ALLOW_ALL
labels:
  deployment-tool: cli-gcloud
name: projects/cloudtutorial-279003/locations/us-central1/functions/hello_world
runtime: python37
serviceAccountEmail: cloudtutorial-279003@appspot.gserviceaccount.com
sourceUploadUrl: https://storage.googleapis.com/gcf-upload-us-central1-aa635b36-c250-4fd0-b45c-1db908086599/ba8df2d2-5ab8-410b-a989-e732981f2eb5.zip?GoogleAccessId=service-227224402169@gcf-admin-robot.iam.gserviceaccount.com&Expires=1592791720&Signature=eiNAlixtKQitC073B2R135QjuqMiUjkrhvtEfaRY3iqiExOTkAMHVhqW0X90Gj8q3Edpu2Be90IfVJbUbusuarb1MHnkmbS9teIw6xLcryXT9SZ%2Fre9gpl%2BY3Sq1TYcv%2BG78KkS%2Ft12APq5XN1%2BUfxgxk%2F79d8SpWXkqrdSSb0%2FzK3fb2%2BEyyIqUA6xlce5TzTiV3spqkBydumBoKY7l5BkVfCy7%2BO94bdTrVinZRrhMuXOm7PLRwuyMvDS3L8uZyp5n7JgdUzEbfMROvQFCPQH%2FfVA%2B8D3HUEUqWtqdTqFS6Pk1jxUrKjtk3eWNeoRZPkWZT4Ux7CiySbJKhHNZNA%3D%3D
st

In [ ]:
# let's check the function info
! gcloud functions describe hello_world

availableMemoryMb: 256
entryPoint: hello_world
httpsTrigger:
  url: https://us-central1-cloudtutorial-279003.cloudfunctions.net/hello_world
ingressSettings: ALLOW_ALL
labels:
  deployment-tool: cli-gcloud
name: projects/cloudtutorial-279003/locations/us-central1/functions/hello_world
runtime: python37
serviceAccountEmail: cloudtutorial-279003@appspot.gserviceaccount.com
sourceUploadUrl: https://storage.googleapis.com/gcf-upload-us-central1-aa635b36-c250-4fd0-b45c-1db908086599/ba8df2d2-5ab8-410b-a989-e732981f2eb5.zip?GoogleAccessId=service-227224402169@gcf-admin-robot.iam.gserviceaccount.com&Expires=1592791720&Signature=eiNAlixtKQitC073B2R135QjuqMiUjkrhvtEfaRY3iqiExOTkAMHVhqW0X90Gj8q3Edpu2Be90IfVJbUbusuarb1MHnkmbS9teIw6xLcryXT9SZ%2Fre9gpl%2BY3Sq1TYcv%2BG78KkS%2Ft12APq5XN1%2BUfxgxk%2F79d8SpWXkqrdSSb0%2FzK3fb2%2BEyyIqUA6xlce5TzTiV3spqkBydumBoKY7l5BkVfCy7%2BO94bdTrVinZRrhMuXOm7PLRwuyMvDS3L8uZyp5n7JgdUzEbfMROvQFCPQH%2FfVA%2B8D3HUEUqWtqdTqFS6Pk1jxUrKjtk3eWNeoRZPkWZT4Ux7CiySbJKhHNZNA%3D%3D
st

In [ ]:
# let's try to trigger the function with python request
import requests

def post_data(url, data=None):
  res = requests.post(url, json=data)
  print("Data has been posted, get response result:  {}".format(res.text))

##### Returned result

As we have passed key with `message` and value with `lu`, so we should get result:`lu`, that's it. 

In [ ]:
hello_url = "https://us-central1-cloudtutorial-279003.cloudfunctions.net/hello_world"

post_data(hello_url, {'message': "lu"})

Data has been posted, get response result:  lu


In [ ]:
# let's try curl
! curl -X post https://us-central1-cloudtutorial-279003.cloudfunctions.net/hello_world -H "Content-Type:application/json" -d '{"message":"lu"}'

lu

In [ ]:
# alright, we have made first cloud function, let's delete it and the source file `main.py`
! gcloud functions delete hello_world

Resource [projects/cloudtutorial-279003/locations/us-central1/function
s/hello_world] will be deleted.

Do you want to continue (Y/n)?  y

Deleted [projects/cloudtutorial-279003/locations/us-central1/functions/hello_world].


In [ ]:
import os

file_to_delete = [x for x in os.listdir('.') if x.endswith('py') ]

for f in file_to_delete:
  print("Now to delete: {}".format(f))
  try:
    os.remove(f)
  except:
    import shutil
    print("Now to remove folder: {}".format(f))
    shutil.rmtree(f)

Now to delete: .config
Now to remove folder: .config
Now to delete: main.py


### Cloud function with Storage

We could define a cloud function to moniter cloud storage bucket, if we have upload or change the files in bucket, we could tigger our function, one thing to notice is the cloud function is event-driven.


In [ ]:
# first let's create a bucket for storing files
! gsutil mb gs://cloud_function_bucket_lugq

Creating gs://cloud_function_bucket_lugq/...


In [ ]:
### let's create a function to print the bucket information
%%writefile main.py

import json

def gcs_func(data, context):
  """
  data is a dictionary contains event info
  contenxt is metadata of trigger event
  """
  # this is som information contain in data and context
  print('Event ID: {}'.format(context.event_id))
  print('Event type: {}'.format(context.event_type))
  print('Bucket: {}'.format(data['bucket']))
  print('File: {}'.format(data['name']))
  print('Metageneration: {}'.format(data['metageneration']))
  print('Created: {}'.format(data['timeCreated']))
  print('Updated: {}'.format(data['updated']))

Overwriting main.py


In [ ]:
# let's deploy our function into cloud, we could set unauthen
! gcloud functions deploy gcs_func --runtime python37 --trigger-resource cloud_function_bucket_lugq --trigger-event google.storage.object.finalize --allow-unauthenticated

availableMemoryMb: 256
entryPoint: gcs_func
eventTrigger:
  eventType: google.storage.object.finalize
  failurePolicy: {}
  resource: projects/_/buckets/cloud_function_bucket_lugq
  service: storage.googleapis.com
ingressSettings: ALLOW_ALL
labels:
  deployment-tool: cli-gcloud
name: projects/cloudtutorial-279003/locations/us-central1/functions/gcs_func
runtime: python37
serviceAccountEmail: cloudtutorial-279003@appspot.gserviceaccount.com
sourceUploadUrl: https://storage.googleapis.com/gcf-upload-us-central1-aa635b36-c250-4fd0-b45c-1db908086599/28835342-ca92-48e3-af2d-79b63bdb0fc4.zip?GoogleAccessId=service-227224402169@gcf-admin-robot.iam.gserviceaccount.com&Expires=1592796457&Signature=aNuUSWhy3L5eROzPIC2QFS9Gbfj%2Blualbc1TRX7flMbHp1jM3Z3OCiXxtdQoDVJFD4jJg09gbX8Z5RJFr54%2FRbmtxiydKOE2PJpuE%2FNlSnJ08gBQPHdsrW9yDLjCr3JW8%2Fu8CbkQhZtbIBRPkC32HPDZI4sfzbSwTnj8de2m4v7sDWXnrCeBnf1ek2sY7KyRhK9%2BKBatUMYZyb3zH9VSbx5pZhCTnx%2BwTmP7JsnpTM%2BDOnHLYQD3N892yvKwGxtvghfItDdmDyRKUAYUYuakbfjjQbXB1v9%

In [ ]:
# as awe have already deployed our function, let's try to trigger this function by uploading a file
# first let's try to create a tempfile 

with open("test.txt", 'w') as f:
  f.write("Test with cloud function")

# let's check
print("files:", os.listdir("."))

# let's upload this file into bucket
! gsutil cp test.txt gs://cloud_function_bucket_lugq

files: ['.config', '.ipynb_checkpoints', 'main.py', 'test.txt', 'adc.json', 'sample_data']
Copying file://test.txt [Content-Type=text/plain]...
/ [1 files][   24.0 B/   24.0 B]                                                
Operation completed over 1 objects/24.0 B.                                       


In [ ]:
# let's get some information about the function result.
! gcloud functions logs read --limit 30


LEVEL  NAME         EXECUTION_ID  TIME_UTC                 LOG
D      hello_world  9opz85hdznvy  2020-06-19 09:47:20.839  Function execution took 71 ms, finished with status code: 200
D      hello_world  9opz8hy33qz3  2020-06-19 09:49:04.260  Function execution started
D      hello_world  9opz8hy33qz3  2020-06-19 09:49:04.272  Function execution took 13 ms, finished with status code: 200
D      hello_world  9opzv9qnpa7a  2020-06-19 09:49:12.111  Function execution started
D      hello_world  9opzv9qnpa7a  2020-06-19 09:49:12.121  Function execution took 11 ms, finished with status code: 200
D      hello_world  9opzt4ju7eig  2020-06-19 09:50:27.804  Function execution started
D      hello_world  9opzt4ju7eig  2020-06-19 09:50:27.836  Function execution took 33 ms, finished with status code: 200
D      hello_world  87wigivkhjuz  2020-06-22 01:39:34.920  Function execution started
D      hello_world  87wigivkhjuz  2020-06-22 01:39:34.939  Function execution took 20 ms, finished with statu

so we do get the result info in cloud function logs about the metadata of our bucket.

##### Delete object in storage

We could also use delete process to trigger our function, but we shouldn't keep the file version on, so that event could be triggered by deletion.

In [ ]:
# lets' redeploy our function with new trigger event
! gcloud functions deploy gcs_func --runtime python37 --trigger-resource cloud_function_bucket_lugq --trigger-event google.storage.object.delete --allow-unauthenticated

availableMemoryMb: 256
entryPoint: gcs_func
eventTrigger:
  eventType: google.storage.object.delete
  failurePolicy: {}
  resource: projects/_/buckets/cloud_function_bucket_lugq
  service: storage.googleapis.com
ingressSettings: ALLOW_ALL
labels:
  deployment-tool: cli-gcloud
name: projects/cloudtutorial-279003/locations/us-central1/functions/gcs_func
runtime: python37
serviceAccountEmail: cloudtutorial-279003@appspot.gserviceaccount.com
sourceUploadUrl: https://storage.googleapis.com/gcf-upload-us-central1-aa635b36-c250-4fd0-b45c-1db908086599/7f8388a7-b25c-4844-89eb-64530e16d62b.zip?GoogleAccessId=service-227224402169@gcf-admin-robot.iam.gserviceaccount.com&Expires=1592797016&Signature=LhkYPby%2FMNaQPuvaToEElusYc5nEw9HaN%2FTMCHWvxLB8aOM%2FiIfFReA8ngzieeCqA%2BrbwStt0jP0OGWdrQpwY7C3AuR4W5RFhyA1zFf6OKg8iagyPXj5deRIZkUpM8tV90WpZG1x0YRgnl6tOjsnQV3Z%2BV6GK0kp9tj%2BFUkNa%2FMIfaAxwH%2Blgx2WLT3aI1GIvL%2F2HB8okVLxRZb5vSbPVKZUaF3qpGCkHbX2tAaWZhAlCVpPVkZClk4xqp1PQKp2ZMx%2BqPVxOFnybDE4HV9WMohJm%2F

In [ ]:
# then let's turn off the version control of the bucket
! gsutil versioning set off gs://cloud_function_bucket_lugq

Suspending versioning for gs://cloud_function_bucket_lugq/...


In [ ]:
# let's change the file content and re-upload the file into bucket
def change_content(file_name='test.txt'):
  import random
  sample_list = ["good", 'machine', 'learning', "gcp", 'feel']
  sample_int = random.randint(0, len(sample_list))
  sample_str = " ".join(random.sample(sample_list, k=sample_int))

  with open(file_name, 'w') as f:
    f.write(sample_str)
  
  print("Random file created.")

def upload_file(file_name='test.txt', bucket_name="cloud_function_bucket_lugq"):
  try:
    os.system("gsutil cp {} gs://{}".format(file_name, bucket_name))
    print("File has been uploaded")
  except:
    raise Exception("When upload file with error,  please retry")

In [ ]:
# first let's change the file content
change_content()

# then upload this file into bucket
upload_file()

Random file created.


In [ ]:
# le's try to remove the file
bucket_name = "cloud_function_bucket_lugq"

! gsutil rm gs://$bucket_name/test.txt

Removing gs://cloud_function_bucket_lugq/test.txt...
/ [1 objects]                                                                   
Operation completed over 1 objects.                                              


In [ ]:
# let's check logs
! gcloud functions logs read --limit 20

# so good, we do trigger our function based on the deletation action.

LEVEL  NAME      EXECUTION_ID  TIME_UTC                 LOG
I      gcs_func  8l1v94pdkp8f  2020-06-22 02:58:46.432  Updated: 2020-06-22T02:58:45.690Z
D      gcs_func  8l1v94pdkp8f  2020-06-22 02:58:46.438  Function execution took 23 ms, finished with status: 'ok'
D      gcs_func  dveb1lcgk5o3  2020-06-22 03:15:10.330  Function execution started
I      gcs_func  dveb1lcgk5o3  2020-06-22 03:15:10.344  Event ID: 1292082177589236
I      gcs_func  dveb1lcgk5o3  2020-06-22 03:15:10.344  Event type: google.storage.object.delete
I      gcs_func  dveb1lcgk5o3  2020-06-22 03:15:10.344  Bucket: cloud_function_bucket_lugq
I      gcs_func  dveb1lcgk5o3  2020-06-22 03:15:10.344  File: test.txt
I      gcs_func  dveb1lcgk5o3  2020-06-22 03:15:10.344  Metageneration: 1
I      gcs_func  dveb1lcgk5o3  2020-06-22 03:15:10.344  Created: 2020-06-22T02:58:45.690Z
I      gcs_func  dveb1lcgk5o3  2020-06-22 03:15:10.344  Updated: 2020-06-22T02:58:45.690Z
D      gcs_func  dveb1lcgk5o3  2020-06-22 03:15:10.345  F

##### File archive with cloud function

In fact, we could do the archive logic for each file in bucket, so we could try to create a event to watch archive process.

In [ ]:
# change the trigger event
! gcloud functions deploy gcs_func --runtime python37 --trigger-resource cloud_function_bucket_lugq --trigger-event google.storage.object.archive --allow-unauthenticated

availableMemoryMb: 256
entryPoint: gcs_func
eventTrigger:
  eventType: google.storage.object.archive
  failurePolicy: {}
  resource: projects/_/buckets/cloud_function_bucket_lugq
  service: storage.googleapis.com
ingressSettings: ALLOW_ALL
labels:
  deployment-tool: cli-gcloud
name: projects/cloudtutorial-279003/locations/us-central1/functions/gcs_func
runtime: python37
serviceAccountEmail: cloudtutorial-279003@appspot.gserviceaccount.com
sourceUploadUrl: https://storage.googleapis.com/gcf-upload-us-central1-aa635b36-c250-4fd0-b45c-1db908086599/3048c844-523f-467b-99e8-18aa18f68d07.zip?GoogleAccessId=service-227224402169@gcf-admin-robot.iam.gserviceaccount.com&Expires=1592797792&Signature=Xc1a%2BR6%2Bp4s6UHV8Hnm14hT7%2BmYJvypARg%2F33Pr93u7rsLwprRwQYsH0V%2B6%2BdtrT%2BgA6jaBYFGvW6pSvs5PftmatzhNlT0VERprVKXtpRC7hAJJNwIEkzehiEz71z3XPizz0XIz3WQbTBjmaXmFnWRl%2Flbo5BuGIozgepT5vYWWN%2B%2Fbr5N%2B5pyn3moPZogi0ixWPn2UPyllklmYe7m%2FBAM4p1q5W8T04K6C275h923nRS1Ev3FqG%2BbVsMzeF0SMyl4lHg8%2BL1D7rSpgP6vJ

In [ ]:
# let's turn the bucket version on
! gsutil versioning set on gs://$bucket_name

# first let's change the file content
change_content()

# then upload this file into bucket
upload_file()

# remove the file to trigger
! gsutil rm gs://$bucket_name/test.txt

# let's check log
! gcloud functions logs read --limit 20

Enabling versioning for gs://cloud_function_bucket_lugq/...
Random file created.
Removing gs://cloud_function_bucket_lugq/test.txt...
/ [1 objects]                                                                   
Operation completed over 1 objects.                                              
LEVEL  NAME      EXECUTION_ID  TIME_UTC                 LOG
I      gcs_func  8l1v94pdkp8f  2020-06-22 02:58:46.432  Updated: 2020-06-22T02:58:45.690Z
D      gcs_func  8l1v94pdkp8f  2020-06-22 02:58:46.438  Function execution took 23 ms, finished with status: 'ok'
D      gcs_func  dveb1lcgk5o3  2020-06-22 03:15:10.330  Function execution started
I      gcs_func  dveb1lcgk5o3  2020-06-22 03:15:10.344  Event ID: 1292082177589236
I      gcs_func  dveb1lcgk5o3  2020-06-22 03:15:10.344  Event type: google.storage.object.delete
I      gcs_func  dveb1lcgk5o3  2020-06-22 03:15:10.344  Bucket: cloud_function_bucket_lugq
I      gcs_func  dveb1lcgk5o3  2020-06-22 03:15:10.344  File: test.txt
I      gcs_fun

##### Metadata change with cloud function

When we change the same file with metadata change, we could also trigger our function.

In [ ]:
# metadata change
! gcloud functions deploy gcs_func --runtime python37 --trigger-resource cloud_function_bucket_lugq --trigger-event google.storage.object.metadataUpdate --allow-unauthenticated

availableMemoryMb: 256
entryPoint: gcs_func
eventTrigger:
  eventType: google.storage.object.metadataUpdate
  failurePolicy: {}
  resource: projects/_/buckets/cloud_function_bucket_lugq
  service: storage.googleapis.com
ingressSettings: ALLOW_ALL
labels:
  deployment-tool: cli-gcloud
name: projects/cloudtutorial-279003/locations/us-central1/functions/gcs_func
runtime: python37
serviceAccountEmail: cloudtutorial-279003@appspot.gserviceaccount.com
sourceUploadUrl: https://storage.googleapis.com/gcf-upload-us-central1-aa635b36-c250-4fd0-b45c-1db908086599/82072c6a-9679-44d4-b4b6-4aea2d5378a8.zip?GoogleAccessId=service-227224402169@gcf-admin-robot.iam.gserviceaccount.com&Expires=1592798111&Signature=fwHi8oulCV65O9GNnhzh%2Be3GJYlMVQqrBIky0O8%2FqqruTWn3fG6ueezgaBCnB24AgWqAooQCO5yAiifkPr5JJjAZN5TVJl4urn7XtKgdt%2Fwu4F3AlsmQmCIelbauagefFT8LvG7lJzJ4ZKHQXoYegd216V6D2dupAsPefzVOu%2BUQ8pKvN%2FGbziJDD4U53boSy9ouDBCtaa5oOaBIzaDXQ4sHYOueBR6ndejNbdNoc5IVSMLiPv8ZwShdIogPWUZhycLhm8ULEErU%2F6Majj1axDGiNffl

In [ ]:
# set versioning to off
! gsutil versioning set off gs://$bucket_name

# first let's change the file content
change_content()

# then upload this file into bucket
upload_file()

# change metadata of files
! gsutil -m setmeta  -h "Content-Type:text/plain" gs://$bucket_name/test.txt

Suspending versioning for gs://cloud_function_bucket_lugq/...
Random file created.
Setting metadata on gs://cloud_function_bucket_lugq/test.txt...
/ [1/1 objects] 100% Done                                                       
Operation completed over 1 objects.                                              
LEVEL  NAME      EXECUTION_ID  TIME_UTC                 LOG
I      gcs_func  dveb1lcgk5o3  2020-06-22 03:15:10.344  Updated: 2020-06-22T02:58:45.690Z
D      gcs_func  dveb1lcgk5o3  2020-06-22 03:15:10.345  Function execution took 17 ms, finished with status: 'ok'
D      gcs_func  dvebyums3h84  2020-06-22 03:16:14.259  Function execution started
I      gcs_func  dvebyums3h84  2020-06-22 03:16:14.267  Event ID: 1292081957882616
I      gcs_func  dvebyums3h84  2020-06-22 03:16:14.268  Event type: google.storage.object.delete
I      gcs_func  dvebyums3h84  2020-06-22 03:16:14.268  Bucket: cloud_function_bucket_lugq
I      gcs_func  dvebyums3h84  2020-06-22 03:16:14.268  File: test.txt
I

In [ ]:
# check logs
! gcloud functions logs read --limit 30

LEVEL  NAME      EXECUTION_ID  TIME_UTC                 LOG
I      gcs_func  dveb1lcgk5o3  2020-06-22 03:15:10.344  Created: 2020-06-22T02:58:45.690Z
I      gcs_func  dveb1lcgk5o3  2020-06-22 03:15:10.344  Updated: 2020-06-22T02:58:45.690Z
D      gcs_func  dveb1lcgk5o3  2020-06-22 03:15:10.345  Function execution took 17 ms, finished with status: 'ok'
D      gcs_func  dvebyums3h84  2020-06-22 03:16:14.259  Function execution started
I      gcs_func  dvebyums3h84  2020-06-22 03:16:14.267  Event ID: 1292081957882616
I      gcs_func  dvebyums3h84  2020-06-22 03:16:14.268  Event type: google.storage.object.delete
I      gcs_func  dvebyums3h84  2020-06-22 03:16:14.268  Bucket: cloud_function_bucket_lugq
I      gcs_func  dvebyums3h84  2020-06-22 03:16:14.268  File: test.txt
I      gcs_func  dvebyums3h84  2020-06-22 03:16:14.268  Metageneration: 1
I      gcs_func  dvebyums3h84  2020-06-22 03:16:14.268  Created: 2020-06-22T03:15:09.619Z
I      gcs_func  dvebyums3h84  2020-06-22 03:16:14.268  U

##### End of cloud function of storage

We have do get process with cloud function for storage, let's remove the function and bucket.

In [ ]:
! gcloud functions delete gcs_func

Resource [projects/cloudtutorial-279003/locations/us-central1/function
s/gcs_func] will be deleted.

Do you want to continue (Y/n)?  y

Deleted [projects/cloudtutorial-279003/locations/us-central1/functions/gcs_func].
Omitting bucket "gs://cloud_function_bucket_lugq/". (Did you mean to do rm -r?)
CommandException: No URLs matched


In [ ]:
! gsutil rm -r gs://$bucket_name

Removing gs://cloud_function_bucket_lugq/test.txt#1592796125268715...
Removing gs://cloud_function_bucket_lugq/test.txt#1592796501028492...
/ [2 objects]                                                                   
Operation completed over 2 objects.                                              
Removing gs://cloud_function_bucket_lugq/...


### Cloud function with Spanner

**Cloud Spanner** is relational database in cloud, just like **MySQL**, but it could handle more data than that with horizontal scale as data becomes more. But it do more expensive than Cloud SQL.

But this tutorial is try to explain how to inter-connect between **Cloud Spanner** and **Cloud function**.

In [ ]:
# define a function to do cleaning for previous logic
import os

def clean():
  file_list = [x for x in os.listdir('.') if x.endswith('py') or x.endswith('txt')]

  for f in file_list:
    print("Now to remove: {}".format(f))
    try:
      os.remove(f)
    except Exception as e:
      print("When to remove: {} with error: {}".format(f, e))

In [ ]:
# before we do anything, first clean the notebook
clean()

Now to remove: main.py
Now to remove: adc.json


#### Spanner preparation

first let's make it work for Spanner before cloud function before we do anything.

In [ ]:
# first create instances
! gcloud spanner instances create lugq-instance --config=regional-us-central1 --description="spanner for cloud" --nodes=1


In [ ]:
# install spanner API
! pip install google-cloud-spanner --quiet

     |████████████████████████████████| 256kB 3.5MB/s 


In [ ]:
# set auth configuration, you could just upload your credential file manually
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = [x for x in os.listdir('.') if x.lower().startswith('cloud')][0]

In [ ]:
# then we should create database
from google.cloud import spanner

project_id = "cloudtutorial-279003"
instance_id = "lugq-instance"
database_id = "cloud-func-db"

spanner_client = spanner.Client(project_id)

table_name = "users"

create_table_sql = """create table {}(user_id int64 not null, first_name string(128), last_name string(128)) primary key (user_id)""".format(table_name)

# init instance
instance = spanner_client.instance(instance_id)
database = instance.database(database_id, ddl_statements=[create_table_sql])

operation = database.create()
print("Waiting table to be created")
operation.result(120)
print("table: {} in database: {} has been created".format(table_name, database_id))

Waiting table to be created
table: users in database: cloud-func-db has been created


In [ ]:
# let's insert some records into this table
insert_sql = "insert {}(user_id, first_name, last_name) values".format(table_name)

# to make the sample data.
insert_values_list = []
for i in range(5):
  insert_values_list.append("({}, '{}', '{}'), ".format(i, "lu_" + str(i), "gq_"  + str(i)))
  if i == 4:
    insert_values_list[-1] = insert_values_list[-1][:-2]

insert_sql = insert_sql + "".join(insert_values_list)

# let's insert values into spanner
def insert_users(transaction):
  res = transaction.execute_update(insert_sql)
  print("{} records inserted".format(res))

database.run_in_transaction(insert_users)

5 records inserted


In [ ]:
# let's make the main.py file
%%writefile main.py
from google.cloud import spanner

project_id = "cloudtutorial-279003"
instance_id = "lugq-instance"
database_id = "cloud-func-db"

client = spanner.Client()
instance = client.instance(instance_id)
database = instance.database(database_id)

def spanner_read_data(request):
  query = "select * from users"

  outputs = []
  with database.snapshot() as snapshot:
    res = snapshot.execute_sql(query)
    
    for row in res:
      output = "user_id: {}, first_name: {}, last_name:{}".format(*row)
      outputs.append(output)
    
  return '\n'.join(outputs)

Overwriting main.py


In [ ]:
# we have to add a requirements.txt file, otherwise will raise error:ImportError: cannot import name 'spanner' from 'google.cloud' (unknown location)
%%writefile requirements.txt
google-cloud-spanner == 1.17.0

Writing requirements.txt


In [ ]:
# let's deploy our code into cloud function
! gcloud functions deploy spanner_read_data --runtime python37 --trigger-http --allow-unauthenticated

availableMemoryMb: 256
entryPoint: spanner_read_data
httpsTrigger:
  url: https://us-central1-cloudtutorial-279003.cloudfunctions.net/spanner_read_data
ingressSettings: ALLOW_ALL
labels:
  deployment-tool: cli-gcloud
name: projects/cloudtutorial-279003/locations/us-central1/functions/spanner_read_data
runtime: python37
serviceAccountEmail: cloudtutorial-279003@appspot.gserviceaccount.com
sourceUploadUrl: https://storage.googleapis.com/gcf-upload-us-central1-aa635b36-c250-4fd0-b45c-1db908086599/67c6502a-bb32-41fd-836d-a006a1be1a50.zip?GoogleAccessId=service-227224402169@gcf-admin-robot.iam.gserviceaccount.com&Expires=1592806276&Signature=j2PUhaxokH2bPTmS%2B%2F7AIBaxvA%2BVkt%2FO8WQwkQFlngDDtJo%2BPw%2B3zu7NlqgIEPSztDlo6gJ7vm3lHBrB%2Bo4Ml6qzigHCVuQo9U%2BqW%2FpRFkh961MQI6Xaeo7EFBU0fv%2BTGzSxM1U%2F3PyfVY90Dph%2BS3iwc2AIW9SKAIlLcjUl621iGdFUO4XYEWX08HWOc0w8J%2BSg%2FvaKbxJYwaz5n7fOKe%2BRWQUTNzR6CVGn8pqErae25G7KEWKiUiaRl0XJ5Gi0Kk0PpatLgzAJKHVtkfwbN7K8av4KstmC%2B7K0qrIFBXlEdEeRQMQ%2ByrpPrK5xUvCKt

In [ ]:
# check spanner version.
spanner.__version__

'1.17.0'

#### Tips

If you face error with function: `Error: could not handle the request`, that's means that your code has error, do check the code first!

In [ ]:
# let's trigger our deployed function, the url is from the above deployed info
deployment_url = "https://us-central1-cloudtutorial-279003.cloudfunctions.net/spanner_read_data"

# so as you see that we do get the result from spanner.
! curl "https://us-central1-cloudtutorial-279003.cloudfunctions.net/spanner_read_data"

user_id: 0, first_name: lu_0, last_name:gq_0
user_id: 1, first_name: lu_1, last_name:gq_1
user_id: 2, first_name: lu_2, last_name:gq_2
user_id: 3, first_name: lu_3, last_name:gq_3
user_id: 4, first_name: lu_4, last_name:gq_4

In [ ]:
# let's try python request 
import requests

res = requests.get(deployment_url)

# with python requests, we get same result
print(res.text)

user_id: 0, first_name: lu_0, last_name:gq_0
user_id: 1, first_name: lu_1, last_name:gq_1
user_id: 2, first_name: lu_2, last_name:gq_2
user_id: 3, first_name: lu_3, last_name:gq_3
user_id: 4, first_name: lu_4, last_name:gq_4


##### Last with Spanner

This parts is for using cloud function to interact with **Cloud Spanner**, so we could do trigger our request by http request.

Let's remove the whole things.

In [ ]:
# remove spanner instances
! gcloud spanner instances delete lugq-instance

Delete instance [lugq-instance]. Are you sure?

Do you want to continue (Y/n)?  y



In [ ]:
# let's remove our functions
! gcloud functions delete spanner_read_data

Resource [projects/cloudtutorial-279003/locations/us-central1/function
s/spanner_read_data] will be deleted.

Do you want to continue (Y/n)?  y

Deleted [projects/cloudtutorial-279003/locations/us-central1/functions/spanner_read_data].


### Cloud PubSub with Cloud function

As I have already created a notebook to explain **Cloud PubSub**, this parts is try to interact **Cloud Functions** and **Cloud PubSub**. 

Let's get start.

In [ ]:
# first let's remove the previous part files.
clean()

Now to remove: main.py
Now to remove: test.txt
Now to remove: requirements.txt


In [ ]:
# before we do anything, we should first create our topic
! gcloud pubsub topics create function_pubub

Created topic [projects/cloudtutorial-279003/topics/function_pubub].


In [ ]:
# let's try to implement logic with pubsub
%%writefile main.py

def pubsub_cloud(event, context):
  """
  `event` is a dictionary contains `message` in `data` key, and
  'attributes' field is for custom attributes.
  `context` is cloud function event metadata, `event_id` is message id,
  'timestamp' is the publish time
  """
  import base64

  print("This function is triggered by messageID: {} published at {}".format(context.event_id, context.timestamp))

  if "data" in event:
    name = base64.b64decode(event['data']).decode('utf-8')
  else:
    name = "LU"
    
  return "Hello {}".format(name) 

Writing main.py


In [ ]:
# let's deploy our function
! gcloud functions deploy pubsub_cloud --runtime python37 --trigger-topic function_pubub

Allow unauthenticated invocations of new function [pubsub_cloud]? 
(y/N)?  y

availableMemoryMb: 256
entryPoint: pubsub_cloud
eventTrigger:
  eventType: google.pubsub.topic.publish
  failurePolicy: {}
  resource: projects/cloudtutorial-279003/topics/function_pubub
  service: pubsub.googleapis.com
ingressSettings: ALLOW_ALL
labels:
  deployment-tool: cli-gcloud
name: projects/cloudtutorial-279003/locations/us-central1/functions/pubsub_cloud
runtime: python37
serviceAccountEmail: cloudtutorial-279003@appspot.gserviceaccount.com
sourceUploadUrl: https://storage.googleapis.com/gcf-upload-us-central1-aa635b36-c250-4fd0-b45c-1db908086599/add00d60-bf3e-4cd6-9721-beb5be2aa952.zip?GoogleAccessId=service-227224402169@gcf-admin-robot.iam.gserviceaccount.com&Expires=1592807155&Signature=Po7W9dZI8tonRn7DVUiGb2FwRbPMNBjMWiO5PJYgSX%2FgSPal6D2hPXfMJ3PuyO4oodDMff%2B8w469XiYdpEUJ%2BWR7W8y5yPezORp8vPzyyN3oa2FK1eoRGuHVuOw9G6P7GJhqb%2Fg7lD%2BF6W3LTvzjLMCl4kH70b3fBHkpFykA0bWqj0oAqiwrsjNT6E8ZR5FCQzpr%2BCwzbT

In [ ]:
# let's try to push some message into the topic: function_pubub
! gcloud pubsub topics publish function_pubub --message '{"data": "GCP"}'

messageIds:
- '1292251120763875'


In [ ]:
# get logs from cloud function
# so you do see that we have triggered our function by pubsub.
! gcloud functions logs read --limit 20

LEVEL  NAME               EXECUTION_ID  TIME_UTC                 LOG
I      gcs_func           2poynb2lwg5y  2020-06-22 03:28:27.515  File: test.txt
I      gcs_func           2poynb2lwg5y  2020-06-22 03:28:27.515  Metageneration: 2
I      gcs_func           2poynb2lwg5y  2020-06-22 03:28:27.515  Created: 2020-06-22T03:28:21.028Z
I      gcs_func           2poynb2lwg5y  2020-06-22 03:28:27.515  Updated: 2020-06-22T03:28:23.089Z
D      gcs_func           2poynb2lwg5y  2020-06-22 03:28:27.520  Function execution took 16 ms, finished with status: 'ok'
D      spanner_read_data  wrnmejbkbc8k  2020-06-22 05:34:44.049  Function execution started
D      spanner_read_data  wrnmejbkbc8k  2020-06-22 05:34:44.706  Function execution took 658 ms, finished with status: 'crash'
D      spanner_read_data  by2p3ajg6zbe  2020-06-22 05:35:05.755  Function execution started
D      spanner_read_data  by2p3ajg6zbe  2020-06-22 05:35:07.525  Function execution took 1771 ms, finished with status: 'crash'
D      s

In [ ]:
# let's try to use python client to publish some message and re-check

# if we directly install pubsub always get error: AttributeError: module 'google.protobuf.descriptor' has no attribute '_internal_create_key'
# we do have to uninstall protobuf first and re-install it. then install pubsub
# then we have to `restart` the runtime!
! pip3 uninstall python3-protobuf --quiet
! pip3 uninstall protobuf --quiet

! pip  install protobuf --quiet

! pip install google-cloud-pubsub --quiet


Proceed (y/n)? y
     |████████████████████████████████| 1.3MB 3.4MB/s 


In [ ]:
from google.cloud import pubsub
import json

publisher_client = pubsub.PublisherClient()

topic_name = "function_pubub"
project_id =  "cloudtutorial-279003"

topic_path = publisher_client.topic_path(project_id, topic_name)

for i in range(3):
  data = "This is {}th message".format(str(i + 1))
  data = {"data": data}
  data = json.dumps(data).encode('utf-8')

  response = publisher_client.publish(topic=topic_path, data=data)
  print("Get response: ", response.result())

Get response:  1292258434722849
Get response:  1292254525131466
Get response:  1292259298932601


In [ ]:
# let's check cloud function logs
# so that we do find another 3 records has triggered our cloud function.
! gcloud functions logs read --limit 50

LEVEL  NAME               EXECUTION_ID  TIME_UTC                 LOG
I      gcs_func           dvebyums3h84  2020-06-22 03:16:14.267  Event ID: 1292081957882616
I      gcs_func           dvebyums3h84  2020-06-22 03:16:14.268  Event type: google.storage.object.delete
I      gcs_func           dvebyums3h84  2020-06-22 03:16:14.268  Bucket: cloud_function_bucket_lugq
I      gcs_func           dvebyums3h84  2020-06-22 03:16:14.268  File: test.txt
I      gcs_func           dvebyums3h84  2020-06-22 03:16:14.268  Metageneration: 1
I      gcs_func           dvebyums3h84  2020-06-22 03:16:14.268  Created: 2020-06-22T03:15:09.619Z
I      gcs_func           dvebyums3h84  2020-06-22 03:16:14.268  Updated: 2020-06-22T03:15:09.619Z
D      gcs_func           dvebyums3h84  2020-06-22 03:16:14.272  Function execution took 14 ms, finished with status: 'ok'
D      gcs_func           epaqsuf604ks  2020-06-22 03:22:08.167  Function execution started
I      gcs_func           epaqsuf604ks  2020-06-22 03:22:

In [ ]:
# let's remove  our cloud function
! gcloud functions delete pubsub_cloud

Resource [projects/cloudtutorial-279003/locations/us-central1/function
s/pubsub_cloud] will be deleted.

Do you want to continue (Y/n)?  y

Deleted [projects/cloudtutorial-279003/locations/us-central1/functions/pubsub_cloud].


In [ ]:
# let's delete our topic
! gcloud pubsub topics delete function_pubub

Deleted topic [projects/cloudtutorial-279003/topics/function_pubub].


### Last words

We have explained how to trigger our **cloud function** with **PubSub**.

In fact, we could use **cloud function** to do many things,  if you are curious about cloud function, you could find more [here]("https://cloud.google.com/functions/docs").